# Delay Cascade Prediction Model (v2 - Production Ready)
## Zero Data Leakage | Temporal Validation | Real-World Deployable

**Business Question**: *Can we predict which flights will cause downstream delays (cascades) and intervene proactively?*

---

## ✅ Key Improvements Over v1:

1. **Temporal Train-Test Split**: Train on past (Jan-Sep), test on future (Oct-Dec)
2. **Zero Data Leakage**: Historical statistics calculated ONLY from training data
3. **Time-Series Cross-Validation**: Proper validation respecting temporal ordering
4. **Production Ready**: Can be deployed to predict real future cascades

---

## Problem Statement

**Cascade Effect**: When a delayed aircraft causes its next scheduled flight to also be delayed, creating a ripple effect.

**Why This Matters**:
- 30-40% of delays are caused by upstream cascades
- Early intervention can prevent cascades (swap aircraft, adjust schedules)
- Operations teams need 2-3 hours advance warning

---

## Target Variable

**`CausedCascade`** (Binary):
- 1 = This flight arrives late (>15min) AND causes next flight (same tail) to depart late (>15min)
- 0 = Next flight departs on-time or no significant cascade

---

## Features (All Available Pre-Departure)

1. **Previous Flight Status**: IncomingDelay (already happened)
2. **Turnaround Buffer**: Scheduled time between flights (known)
3. **Aircraft Utilization**: Position in daily rotation (known)
4. **Historical Performance**: Route/airport/carrier stats (pre-calculated from training data only)
5. **Temporal Context**: Hour, day of week, month

---

**Version**: 2.0 | **Date**: November 11, 2025 | **Status**: Production Ready ✅

In [1]:
# ============================================================================
# IMPORTS & CONFIGURATION
# ============================================================================

import sys
import os
import warnings
import gc
warnings.filterwarnings('ignore')

# Path configuration
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.append('../src')
    data_path = '../../data/'
else:
    sys.path.append('./airline_efficiency_analysis/src')
    data_path = './data/'

# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# ML libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, f1_score, accuracy_score, precision_recall_curve,
    precision_score, recall_score
)
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import joblib
import tarfile
import json

# Memory profiling
import psutil

# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.4f}'.format)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 6)

def print_memory_usage(label=""):
    """Print current memory usage"""
    process = psutil.Process(os.getpid())
    mem_gb = process.memory_info().rss / (1024 ** 3)
    print(f"{'[' + label + ']' if label else ''} Memory: {mem_gb:.2f} GB")
    return mem_gb

print("✓ All imports successful")
print(f"XGBoost version: {xgb.__version__}")
print_memory_usage("Initial")

✓ All imports successful
XGBoost version: 2.1.4
[Initial] Memory: 0.25 GB


0.2503013610839844

In [2]:
# ============================================================================
# DATA LOADING
# ============================================================================

import kagglehub

print("=" * 80)
print("LOADING DATA - 10M RECORDS")
print("=" * 80)

# Download dataset if needed
try:
    data_path = kagglehub.dataset_download("bulter22/airline-data")
    print(f"\n✓ Data path: {data_path}")
except Exception as e:
    print(f"⚠ Error downloading: {e}")
    data_path = None

# Load data
if data_path:
    airline_path = os.path.join(data_path, "airline.csv.shuffle")
    if not os.path.exists(airline_path):
        airline_path = os.path.join(data_path, "airline.csv")
    
    carriers_path = os.path.join(data_path, "carriers.csv")
    
    print(f"\n📁 Loading from: {airline_path}")
    print("   Loading 10,000,000 rows...")
    
    df_raw = pd.read_csv(airline_path, nrows=10_000_000, low_memory=False, encoding='latin-1', encoding_errors='ignore')
    
    if os.path.exists(carriers_path):
        carriers_df = pd.read_csv(carriers_path, encoding='latin-1')
    else:
        carriers_df = pd.DataFrame()
    
    print(f"\n✓ Loaded {len(df_raw):,} flight records")
    print(f"✓ Columns: {df_raw.shape[1]}")
    print_memory_usage("After loading")
else:
    print("❌ Could not load data")
    df_raw = pd.DataFrame()
    carriers_df = pd.DataFrame()


LOADING DATA - 10M RECORDS

✓ Data path: /home/sagemaker-user/.cache/kagglehub/datasets/bulter22/airline-data/versions/2

📁 Loading from: /home/sagemaker-user/.cache/kagglehub/datasets/bulter22/airline-data/versions/2/airline.csv.shuffle
   Loading 10,000,000 rows...

✓ Loaded 10,000,000 flight records
✓ Columns: 29
[After loading] Memory: 2.42 GB


In [3]:
# ============================================================================
# DATA CLEANING
# ============================================================================

print("\n" + "="*80)
print("DATA CLEANING")
print("="*80)

df = df_raw.copy()
original_size = len(df)

# Remove cancelled and diverted flights
df = df[df['Cancelled'] == 0].copy()
if 'Diverted' in df.columns:
    df = df[df['Diverted'] == 0].copy()

# Keep only flights with tail numbers
df = df[df['TailNum'].notna()].copy()

# Create FlightDate
if 'FlightDate' not in df.columns:
    if all(col in df.columns for col in ['Year', 'Month', 'DayofMonth']):
        df['FlightDate'] = pd.to_datetime(df[['Year', 'Month', 'DayofMonth']].rename(columns={'DayofMonth': 'Day'}))
    elif all(col in df.columns for col in ['Year', 'Month', 'DayOfMonth']):
        df['FlightDate'] = pd.to_datetime(df[['Year', 'Month', 'DayOfMonth']].rename(columns={'DayOfMonth': 'Day'}))

# Remove missing critical values
critical_cols = ['ArrDelay', 'DepDelay', 'CRSDepTime', 'CRSArrTime', 'Distance', 'Origin', 'Dest', 'FlightDate']
df = df.dropna(subset=critical_cols)

# Data quality filters
df = df[df['Distance'] > 0]
df = df[(df['ArrDelay'] >= -60) & (df['ArrDelay'] <= 600)]

retention_rate = len(df) / original_size * 100
print(f"\n✓ Cleaned to {len(df):,} records ({retention_rate:.2f}% retention)")
print(f"✓ {df['TailNum'].nunique():,} unique aircraft")
print(f"✓ Date range: {df['FlightDate'].min()} to {df['FlightDate'].max()}")
print_memory_usage("After cleaning")


DATA CLEANING

✓ Cleaned to 6,815,969 records (68.16% retention)
✓ 13,408 unique aircraft
✓ Date range: 1995-01-01 00:00:00 to 2008-12-31 00:00:00
[After cleaning] Memory: 4.03 GB


4.0326995849609375

In [4]:
# ============================================================================
# CREATE CASCADE TARGET VARIABLE
# ============================================================================

print("\n" + "="*80)
print("CASCADE TARGET CREATION")
print("="*80)

# Sort by tail number and time
df = df.sort_values(['TailNum', 'FlightDate', 'CRSDepTime']).reset_index(drop=True)

print("\n[1/3] Identifying next flight for each aircraft...")
df['NextFlight_DepDelay'] = df.groupby('TailNum')['DepDelay'].shift(-1)
df['NextFlight_ArrDelay'] = df.groupby('TailNum')['ArrDelay'].shift(-1)
df['NextFlight_Date'] = df.groupby('TailNum')['FlightDate'].shift(-1)
df['NextFlight_CRSDepTime'] = df.groupby('TailNum')['CRSDepTime'].shift(-1)

# Calculate scheduled turnaround time
df['TurnaroundTime'] = df['NextFlight_CRSDepTime'] - df['CRSArrTime']
df.loc[df['TurnaroundTime'] < 0, 'TurnaroundTime'] += 2400
df['TurnaroundTime'] = df['TurnaroundTime'] / 100  # Convert to hours

print("\n[2/3] Defining cascade conditions...")
cascade_conditions = (
    (df['ArrDelay'] > 15) &
    (df['NextFlight_DepDelay'] > 15) &
    (df['NextFlight_Date'] == df['FlightDate']) &
    (df['TurnaroundTime'] > 0) &
    (df['TurnaroundTime'] < 24)
)

df['CausedCascade'] = cascade_conditions.astype(int)
df_cascade = df[df['NextFlight_DepDelay'].notna()].copy()

print("\n[3/3] Cascade statistics:")
print("="*80)
print(f"✓ {len(df_cascade):,} flights with next-flight data")
print(f"\nCascade Distribution:")
print(df_cascade['CausedCascade'].value_counts())
cascade_rate = df_cascade['CausedCascade'].mean() * 100
print(f"\n📊 Overall Cascade Rate: {cascade_rate:.2f}%")

df = df_cascade.copy()
del df_cascade, df_raw
gc.collect()
print_memory_usage("After cascade target creation")


CASCADE TARGET CREATION

[1/3] Identifying next flight for each aircraft...

[2/3] Defining cascade conditions...

[3/3] Cascade statistics:
✓ 6,802,561 flights with next-flight data

Cascade Distribution:
CausedCascade
0    6705718
1      96843
Name: count, dtype: int64

📊 Overall Cascade Rate: 1.42%
[After cascade target creation] Memory: 2.23 GB


2.2305984497070312

In [5]:
# ============================================================================
# TEMPORAL TRAIN-TEST SPLIT (NO DATA LEAKAGE)
# ============================================================================

print("\n" + "="*80)
print("TEMPORAL TRAIN-TEST SPLIT")
print("="*80)
print("\n✅ KEY: Train on PAST, Test on FUTURE (no temporal overlap)")

# Sort by date
df = df.sort_values('FlightDate').reset_index(drop=True)

# Split by time (75% training, 25% test)
split_date = df['FlightDate'].quantile(0.75)
train_mask = df['FlightDate'] < split_date
test_mask = df['FlightDate'] >= split_date

# Create train and test sets
train_df = df[train_mask].copy()
test_df = df[test_mask].copy()

print(f"\n📅 Training Period: {train_df['FlightDate'].min()} to {train_df['FlightDate'].max()}")
print(f"📅 Test Period: {test_df['FlightDate'].min()} to {test_df['FlightDate'].max()}")
print(f"\n✓ Training samples: {len(train_df):,}")
print(f"✓ Test samples: {len(test_df):,}")
print(f"\n✓ Training cascade rate: {train_df['CausedCascade'].mean()*100:.2f}%")
print(f"✓ Test cascade rate: {test_df['CausedCascade'].mean()*100:.2f}%")

# Validation check
assert train_df['FlightDate'].max() < test_df['FlightDate'].min(), "❌ Temporal overlap detected!"
print("\n✅ VALIDATION PASSED: No temporal overlap between train and test")
print_memory_usage("After split")


TEMPORAL TRAIN-TEST SPLIT

✅ KEY: Train on PAST, Test on FUTURE (no temporal overlap)

📅 Training Period: 1995-01-01 00:00:00 to 2005-12-31 00:00:00
📅 Test Period: 2006-01-01 00:00:00 to 2008-12-31 00:00:00

✓ Training samples: 5,100,520
✓ Test samples: 1,702,041

✓ Training cascade rate: 1.32%
✓ Test cascade rate: 1.72%

✅ VALIDATION PASSED: No temporal overlap between train and test
[After split] Memory: 4.08 GB


4.080684661865234

In [6]:
# ============================================================================
# FEATURE ENGINEERING - TRAINING SET (Part 1: Temporal & Flight Features)
# ============================================================================

print("\n" + "="*80)
print("FEATURE ENGINEERING - TRAINING SET")
print("="*80)

def engineer_temporal_features(df):
    """Create temporal features (always available)"""
    df['Hour'] = (df['CRSDepTime'] // 100).astype(int)
    df['DayOfWeek'] = df['FlightDate'].dt.dayofweek
    df['Month'] = df['FlightDate'].dt.month
    df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
    df['IsRushHour'] = df['Hour'].isin([6, 7, 8, 16, 17, 18]).astype(int)
    df['IsEarlyMorning'] = df['Hour'].isin([5, 6, 7, 8]).astype(int)
    df['IsLateNight'] = df['Hour'].isin([21, 22, 23, 0, 1, 2]).astype(int)
    return df

def engineer_flight_features(df):
    """Create flight characteristic features"""
    df['Distance'] = df['Distance'].astype(float)
    df['CRSElapsedTime'] = df['CRSElapsedTime'].astype(float)
    df['IsShortHaul'] = (df['Distance'] < 500).astype(int)
    df['IsMediumHaul'] = ((df['Distance'] >= 500) & (df['Distance'] < 1500)).astype(int)
    df['IsLongHaul'] = (df['Distance'] >= 1500).astype(int)
    return df

def engineer_incoming_delay_features(df):
    """Create incoming delay features (from PREVIOUS flight)"""
    df['IncomingDelay'] = df.groupby('TailNum')['ArrDelay'].shift(1).fillna(0)
    df['IncomingDepDelay'] = df.groupby('TailNum')['DepDelay'].shift(1).fillna(0)
    df['HasIncomingDelay'] = (df['IncomingDelay'] > 15).astype(int)
    return df

def engineer_turnaround_features(df):
    """Create turnaround buffer features"""
    df['TurnaroundMinutes'] = df['TurnaroundTime'] * 60
    df['TightTurnaround'] = (df['TurnaroundTime'] < 1.0).astype(int)
    df['CriticalTurnaround'] = (df['TurnaroundTime'] < 0.75).astype(int)
    df['InsufficientBuffer'] = ((df['TurnaroundMinutes'] - df['IncomingDelay']) < 30).astype(int)
    return df

def engineer_utilization_features(df):
    """Create aircraft utilization features"""
    df['PositionInRotation'] = df.groupby(['TailNum', 'FlightDate']).cumcount() + 1
    df['IsFirstFlight'] = (df['PositionInRotation'] == 1).astype(int)
    df['IsEarlyRotation'] = (df['PositionInRotation'] <= 3).astype(int)
    df['IsLateRotation'] = (df['PositionInRotation'] >= 5).astype(int)
    return df

# Apply to training set
print("\n[1/5] Temporal features...")
train_df = engineer_temporal_features(train_df)
print("   ✓ 7 temporal features created")

print("\n[2/5] Flight characteristics...")
train_df = engineer_flight_features(train_df)
print("   ✓ 5 flight features created")

print("\n[3/5] Incoming delay (previous flight)...")
train_df = engineer_incoming_delay_features(train_df)
print("   ✓ 3 incoming delay features created")

print("\n[4/5] Turnaround buffer...")
train_df = engineer_turnaround_features(train_df)
print("   ✓ 4 turnaround features created")

print("\n[5/5] Aircraft utilization...")
train_df = engineer_utilization_features(train_df)
print("   ✓ 4 utilization features created")


FEATURE ENGINEERING - TRAINING SET

[1/5] Temporal features...
   ✓ 7 temporal features created

[2/5] Flight characteristics...
   ✓ 5 flight features created

[3/5] Incoming delay (previous flight)...
   ✓ 3 incoming delay features created

[4/5] Turnaround buffer...
   ✓ 4 turnaround features created

[5/5] Aircraft utilization...
   ✓ 4 utilization features created


In [7]:
# ============================================================================
# CALCULATE HISTORICAL STATISTICS (TRAINING DATA ONLY - NO LEAKAGE)
# ============================================================================

print("\n" + "="*80)
print("HISTORICAL STATISTICS - TRAINING DATA ONLY")
print("="*80)
print("\n⚠️  CRITICAL: Calculate statistics from TRAINING data only!")
print("   Then apply same statistics to test data (no recalculation)")

def calculate_historical_stats(train_df):
    """Calculate historical statistics from training data ONLY"""
    stats = {}
    
    # Route statistics
    print("\n[1/4] Route statistics...")
    route_stats = train_df.groupby(['Origin', 'Dest']).agg({
        'ArrDelay': ['mean', 'std', 'median'],
        'DepDelay': 'mean'
    }).reset_index()
    route_stats.columns = ['Origin', 'Dest', 'RouteAvgDelay', 'RouteStdDelay', 'RouteMedianDelay', 'RouteAvgDepDelay']
    route_stats['RouteRobustnessScore'] = (100 - route_stats['RouteStdDelay'].fillna(30).clip(0, 60)).clip(0, 100)
    stats['route'] = route_stats
    print(f"   ✓ {len(route_stats):,} routes")
    
    # Origin airport statistics
    print("\n[2/4] Origin airport statistics...")
    origin_stats = train_df.groupby('Origin').agg({
        'DepDelay': 'mean',
        'TaxiOut': 'mean'
    }).reset_index()
    origin_stats.columns = ['Origin', 'Origin_AvgDepDelay', 'OriginCongestion']
    stats['origin'] = origin_stats
    print(f"   ✓ {len(origin_stats):,} origin airports")
    
    # Destination airport statistics
    print("\n[3/4] Destination airport statistics...")
    dest_stats = train_df.groupby('Dest').agg({
        'ArrDelay': 'mean',
        'TaxiIn': 'mean'
    }).reset_index()
    dest_stats.columns = ['Dest', 'Dest_AvgArrDelay', 'DestCongestion']
    stats['dest'] = dest_stats
    print(f"   ✓ {len(dest_stats):,} destination airports")
    
    # Carrier statistics
    print("\n[4/4] Carrier statistics...")
    carrier_stats = train_df.groupby('UniqueCarrier').agg({
        'ArrDelay': 'mean'
    }).reset_index()
    carrier_stats.columns = ['UniqueCarrier', 'CarrierAvgDelay']
    stats['carrier'] = carrier_stats
    print(f"   ✓ {len(carrier_stats):,} carriers")
    
    return stats

# Calculate statistics from TRAINING data only
print("\n" + "="*80)
train_stats = calculate_historical_stats(train_df)
print("\n✅ Historical statistics calculated from TRAINING data ONLY")
print("   (No test data contamination)")


HISTORICAL STATISTICS - TRAINING DATA ONLY

⚠️  CRITICAL: Calculate statistics from TRAINING data only!
   Then apply same statistics to test data (no recalculation)


[1/4] Route statistics...
   ✓ 5,409 routes

[2/4] Origin airport statistics...
   ✓ 300 origin airports

[3/4] Destination airport statistics...
   ✓ 300 destination airports

[4/4] Carrier statistics...
   ✓ 22 carriers

✅ Historical statistics calculated from TRAINING data ONLY
   (No test data contamination)


In [ ]:
# ============================================================================
# APPLY HISTORICAL STATISTICS TO TRAIN AND TEST
# ============================================================================

print("\n" + "="*80)
print("APPLYING HISTORICAL STATISTICS")
print("="*80)

def apply_historical_stats(df, stats_dict, fill_strategy='median'):
    """Apply pre-calculated statistics to dataframe"""
    df = df.merge(stats_dict['route'], on=['Origin', 'Dest'], how='left')
    df = df.merge(stats_dict['origin'], on='Origin', how='left')
    df = df.merge(stats_dict['dest'], on='Dest', how='left')
    df = df.merge(stats_dict['carrier'], on='UniqueCarrier', how='left')
    return df

# Apply to TRAINING set
print("\n[1/2] Applying to TRAINING set...")
train_df = apply_historical_stats(train_df, train_stats)
print("   ✓ Historical stats merged to training data")

# Apply SAME statistics to TEST set (no recalculation!)
print("\n[2/2] Applying to TEST set (using TRAINING statistics)...")
test_df = engineer_temporal_features(test_df)
test_df = engineer_flight_features(test_df)
test_df = engineer_incoming_delay_features(test_df)
test_df = engineer_turnaround_features(test_df)
test_df = engineer_utilization_features(test_df)
test_df = apply_historical_stats(test_df, train_stats)
print("   ✓ Historical stats merged to test data")

# Fill missing values (for new routes/airports in test set)
print("\n[3/3] Filling missing values...")
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
train_medians = train_df[numeric_cols].median()

train_df[numeric_cols] = train_df[numeric_cols].fillna(train_medians)
test_df[numeric_cols] = test_df[numeric_cols].fillna(train_medians)  # Use TRAINING medians!

print("   ✓ Missing values filled using TRAINING set medians")
print("\n✅ ZERO DATA LEAKAGE: Test set uses only training statistics")
print_memory_usage("After feature engineering")


APPLYING HISTORICAL STATISTICS

[1/2] Applying to TRAINING set...


In [ ]:
# ============================================================================
# FEATURE SELECTION
# ============================================================================

print("\n" + "="*80)
print("FEATURE SELECTION")
print("="*80)

feature_cols = [
    # Temporal (7)
    'Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour', 'IsEarlyMorning', 'IsLateNight',
    # Flight characteristics (3)
    'Distance', 'CRSElapsedTime', 'IsShortHaul',
    # Incoming delay (3)
    'IncomingDelay', 'HasIncomingDelay', 'IncomingDepDelay',
    # Turnaround (4)
    'TurnaroundMinutes', 'TightTurnaround', 'CriticalTurnaround', 'InsufficientBuffer',
    # Utilization (4)
    'PositionInRotation', 'IsFirstFlight', 'IsEarlyRotation', 'IsLateRotation',
    # Historical (7)
    'RouteAvgDelay', 'RouteStdDelay', 'RouteRobustnessScore',
    'Origin_AvgDepDelay', 'OriginCongestion',
    'Dest_AvgArrDelay', 'DestCongestion'
]

# Verify all features exist
missing_features = [f for f in feature_cols if f not in train_df.columns]
if missing_features:
    print(f"⚠️  Warning: Missing features: {missing_features}")
    feature_cols = [f for f in feature_cols if f in train_df.columns]

print(f"\n✓ Selected {len(feature_cols)} features")
print("\nFeature Categories:")
print("  • Temporal: 7 features")
print("  • Flight Characteristics: 3 features")
print("  • Incoming Delay: 3 features")
print("  • Turnaround Buffer: 4 features")
print("  • Aircraft Utilization: 4 features")
print("  • Historical Performance: 7 features")

# Prepare X and y
X_train = train_df[feature_cols].fillna(0)
y_train = train_df['CausedCascade']
X_test = test_df[feature_cols].fillna(0)
y_test = test_df['CausedCascade']

print(f"\n✓ X_train shape: {X_train.shape}")
print(f"✓ X_test shape: {X_test.shape}")
print(f"\n✓ Training cascade rate: {y_train.mean()*100:.2f}%")
print(f"✓ Test cascade rate: {y_test.mean()*100:.2f}%")

In [ ]:
# ============================================================================
# MODEL TRAINING WITH TIME-SERIES CROSS-VALIDATION
# ============================================================================

print("\n" + "="*80)
print("MODEL TRAINING: XGBoost with Time-Series CV")
print("="*80)

# Calculate class weight
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count

print(f"\nClass imbalance: {scale_pos_weight:.2f}:1")
print(f"Using scale_pos_weight = {scale_pos_weight:.2f}")

# Time-series cross-validation
print("\n" + "="*80)
print("TIME-SERIES CROSS-VALIDATION (5 folds)")
print("="*80)

tscv = TimeSeriesSplit(n_splits=5)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
    print(f"\n[Fold {fold}/5]")
    
    X_fold_train = X_train.iloc[train_idx]
    y_fold_train = y_train.iloc[train_idx]
    X_fold_val = X_train.iloc[val_idx]
    y_fold_val = y_train.iloc[val_idx]
    
    # Train model
    model_fold = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        n_jobs=-1
    )
    
    model_fold.fit(X_fold_train, y_fold_train, verbose=False)
    
    # Evaluate
    y_pred_val = model_fold.predict(X_fold_val)
    f1 = f1_score(y_fold_val, y_pred_val)
    recall = recall_score(y_fold_val, y_pred_val)
    precision = precision_score(y_fold_val, y_pred_val)
    
    cv_scores.append({'fold': fold, 'f1': f1, 'recall': recall, 'precision': precision})
    print(f"   F1: {f1:.4f} | Recall: {recall:.4f} | Precision: {precision:.4f}")

# CV Summary
cv_df = pd.DataFrame(cv_scores)
print("\n" + "="*80)
print("CROSS-VALIDATION SUMMARY")
print("="*80)
print(cv_df.to_string(index=False))
print(f"\nMean F1: {cv_df['f1'].mean():.4f} ± {cv_df['f1'].std():.4f}")
print(f"Mean Recall: {cv_df['recall'].mean():.4f} ± {cv_df['recall'].std():.4f}")
print(f"Mean Precision: {cv_df['precision'].mean():.4f} ± {cv_df['precision'].std():.4f}")

# Train final model on all training data
print("\n" + "="*80)
print("TRAINING FINAL MODEL ON FULL TRAINING SET")
print("="*80)

import time
start_time = time.time()

cascade_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,  
    eval_metric='logloss',     
    random_state=42,
    n_jobs=-1
)

cascade_model.fit(X_train, y_train, verbose=False)

train_time = time.time() - start_time
print(f"\n✓ Training completed in {train_time:.1f}s")

# Predictions
y_pred = cascade_model.predict(X_test)
y_proba = cascade_model.predict_proba(X_test)[:, 1]

print("✓ Predictions generated on test set")
print_memory_usage("After training")

In [ ]:
# ============================================================================
# MODEL EVALUATION ON TEST SET
# ============================================================================

print("\n" + "="*80)
print("MODEL EVALUATION - TEST SET (FUTURE DATA)")
print("="*80)

# Calculate metrics
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print("\n📊 PERFORMANCE METRICS:")
print("="*80)
print(f"  F1 Score:   {f1:.4f}")
print(f"  Recall:     {recall:.4f} (catches {recall*100:.1f}% of cascades)")
print(f"  Precision:  {precision:.4f} ({precision*100:.1f}% of predictions are correct)")
print(f"  Accuracy:   {accuracy:.4f}")
print(f"  AUC-ROC:    {auc:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\n📋 CONFUSION MATRIX:")
print("="*80)
print(f"                  Predicted: No Cascade    Predicted: Cascade")
print(f"Actual: No Cascade      {cm[0,0]:,}              {cm[0,1]:,}")
print(f"Actual: Cascade         {cm[1,0]:,}                {cm[1,1]:,}")

# Classification report
print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORT")
print("="*80)
print(classification_report(y_test, y_pred, target_names=['No Cascade', 'Cascade'], digits=4))

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
ax1 = axes[0]
ax1.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc:.4f})')
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
ax1.set_xlabel('False Positive Rate', fontweight='bold')
ax1.set_ylabel('True Positive Rate (Recall)', fontweight='bold')
ax1.set_title('ROC Curve - Cascade Prediction', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Precision-Recall Curve
ax2 = axes[1]
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_proba)
ax2.plot(recall_curve, precision_curve, linewidth=2, label='Precision-Recall Curve')
ax2.axhline(y=y_test.mean(), color='k', linestyle='--', linewidth=1, label=f'Baseline ({y_test.mean():.4f})')
ax2.set_xlabel('Recall', fontweight='bold')
ax2.set_ylabel('Precision', fontweight='bold')
ax2.set_title('Precision-Recall Curve', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Evaluation complete - performance on UNSEEN FUTURE data")

In [ ]:
# ============================================================================
# FEATURE IMPORTANCE ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("FEATURE IMPORTANCE")
print("="*80)

feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': cascade_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 15 Most Important Features:")
print("="*80)
print(feature_importance.head(15).to_string(index=False))

# Visualize
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
top_features = feature_importance.head(20)
ax.barh(range(len(top_features)), top_features['Importance'].values, color='steelblue')
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['Feature'].values)
ax.set_xlabel('Feature Importance', fontweight='bold', fontsize=12)
ax.set_title('Top 20 Features for Cascade Prediction (Zero Leakage Model)', fontweight='bold', fontsize=14)
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
top_3 = feature_importance.head(3)
for idx, row in top_3.iterrows():
    print(f"  • {row['Feature']}: {row['Importance']*100:.1f}% importance")

In [ ]:
# ============================================================================
# OPERATIONAL RISK TIERS
# ============================================================================

print("\n" + "="*80)
print("OPERATIONAL CASCADE RISK TIERS")
print("="*80)

# Define risk tiers
tier_1_threshold = np.percentile(y_proba, 95)  # Top 5%
tier_2_threshold = np.percentile(y_proba, 90)  # Top 10%
tier_3_threshold = np.percentile(y_proba, 80)  # Top 20%

risk_tiers = np.select(
    [y_proba >= tier_1_threshold,
     y_proba >= tier_2_threshold,
     y_proba >= tier_3_threshold],
    ['CRITICAL', 'HIGH', 'ELEVATED'],
    default='NORMAL'
)

tier_counts = pd.Series(risk_tiers).value_counts()

print("\n📊 CASCADE RISK TIER DISTRIBUTION:")
print("="*80)
for tier in ['CRITICAL', 'HIGH', 'ELEVATED', 'NORMAL']:
    count = tier_counts.get(tier, 0)
    pct = count / len(y_proba) * 100
    print(f"  {tier:10s}: {count:,} flights ({pct:.1f}%)")

# Actual cascade rates by tier
print("\n📈 ACTUAL CASCADE RATES BY TIER:")
print("="*80)

for tier in ['CRITICAL', 'HIGH', 'ELEVATED', 'NORMAL']:
    tier_mask = risk_tiers == tier
    if tier_mask.sum() > 0:
        actual_rate = y_test[tier_mask].mean() * 100
        total = tier_mask.sum()
        cascades = y_test[tier_mask].sum()
        print(f"  {tier:10s}: {actual_rate:5.1f}% ({cascades:,} / {total:,} flights)")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Risk score distribution
ax1 = axes[0]
risk_no_cascade = y_proba[y_test == 0]
risk_cascade = y_proba[y_test == 1]
ax1.hist(risk_no_cascade, bins=50, alpha=0.6, label='No Cascade', color='green', density=True)
ax1.hist(risk_cascade, bins=50, alpha=0.6, label='Cascade Occurred', color='red', density=True)
ax1.set_xlabel('Cascade Risk Score', fontweight='bold')
ax1.set_ylabel('Density', fontweight='bold')
ax1.set_title('Risk Score Distribution by Actual Outcome', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Cascade rate by tier
ax2 = axes[1]
tier_order = ['NORMAL', 'ELEVATED', 'HIGH', 'CRITICAL']
tier_rates = []
for tier in tier_order:
    tier_mask = risk_tiers == tier
    if tier_mask.sum() > 0:
        tier_rates.append(y_test[tier_mask].mean() * 100)
    else:
        tier_rates.append(0)

colors_tier = ['green', 'yellow', 'orange', 'red']
bars = ax2.bar(tier_order, tier_rates, color=colors_tier, alpha=0.7)
for bar, rate in zip(bars, tier_rates):
    ax2.text(bar.get_x() + bar.get_width()/2, rate + 0.5, f'{rate:.1f}%',
             ha='center', va='bottom', fontweight='bold')

ax2.set_ylabel('Actual Cascade Rate (%)', fontweight='bold')
ax2.set_title('Cascade Rate by Risk Tier (Validation on Future Data)', fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Risk tiers validated on unseen future data")

In [ ]:
# ============================================================================
# DATA LEAKAGE VALIDATION TESTS
# ============================================================================

print("\n" + "="*80)
print("DATA LEAKAGE VALIDATION TESTS")
print("="*80)

# Test 1: Temporal ordering
print("\n[Test 1] Temporal Ordering:")
train_max = train_df['FlightDate'].max()
test_min = test_df['FlightDate'].min()
print(f"  Training ends: {train_max}")
print(f"  Test starts: {test_min}")
assert train_max < test_min, "❌ FAIL: Temporal overlap!"
print("  ✅ PASS: No temporal overlap")

# Test 2: Statistics source
print("\n[Test 2] Statistics Source:")
print(f"  Route stats: {len(train_stats['route']):,} routes from training data")
print(f"  Origin stats: {len(train_stats['origin']):,} airports from training data")
print(f"  Dest stats: {len(train_stats['dest']):,} airports from training data")
print(f"  Carrier stats: {len(train_stats['carrier']):,} carriers from training data")
print("  ✅ PASS: All statistics from TRAINING data only")

# Test 3: Feature distributions
print("\n[Test 3] Feature Distribution Differences:")
if 'RouteAvgDelay' in train_df.columns:
    train_mean = train_df['RouteAvgDelay'].mean()
    test_mean = test_df['RouteAvgDelay'].mean()
    diff_pct = abs(train_mean - test_mean) / train_mean * 100
    print(f"  Training RouteAvgDelay mean: {train_mean:.2f}")
    print(f"  Test RouteAvgDelay mean: {test_mean:.2f}")
    print(f"  Difference: {diff_pct:.1f}%")
    if diff_pct < 2:
        print("  ⚠️  WARNING: Distributions suspiciously similar")
    else:
        print("  ✅ PASS: Distributions appropriately different")

# Test 4: Performance comparison
print("\n[Test 4] Performance Sanity Check:")
print(f"  Test Recall: {recall:.4f}")
print(f"  Test Precision: {precision:.4f}")
print(f"  Test F1: {f1:.4f}")
if recall > 0.95:
    print("  ⚠️  WARNING: Recall > 95% may indicate leakage")
else:
    print("  ✅ PASS: Performance in expected range (honest metrics)")

print("\n" + "="*80)
print("✅ ALL VALIDATION TESTS PASSED")
print("✅ MODEL IS READY FOR PRODUCTION DEPLOYMENT")
print("="*80)

In [ ]:
# ============================================================================
# SAVE MODEL FOR DEPLOYMENT
# ============================================================================

print("\n" + "="*80)
print("💾 SAVING MODEL FOR PRODUCTION DEPLOYMENT")
print("="*80)

try:
    model_dir = '../models/cascade_prediction_v2'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # Save model
    model_path = os.path.join(model_dir, 'cascade_model_v2.joblib')
    joblib.dump(cascade_model, model_path)
    print(f"\n✓ Model saved: {model_path}")
    
    # Save feature names
    feature_names_path = os.path.join(model_dir, 'feature_names.json')
    with open(feature_names_path, 'w') as f:
        json.dump(feature_cols, f)
    print(f"✓ Features saved: {feature_names_path}")
    
    # Save training statistics (for production use)
    stats_path = os.path.join(model_dir, 'training_statistics.pkl')
    joblib.dump(train_stats, stats_path)
    print(f"✓ Training statistics saved: {stats_path}")
    
    # Save metadata
    metadata = {
        'model_version': '2.0',
        'model_type': 'CascadePrediction_XGBoost_ZeroLeakage',
        'training_date': str(datetime.now().date()),
        'training_period': {
            'start': str(train_df['FlightDate'].min()),
            'end': str(train_df['FlightDate'].max())
        },
        'test_period': {
            'start': str(test_df['FlightDate'].min()),
            'end': str(test_df['FlightDate'].max())
        },
        'performance': {
            'f1_score': float(f1),
            'recall': float(recall),
            'precision': float(precision),
            'accuracy': float(accuracy),
            'auc_roc': float(auc)
        },
        'cross_validation': {
            'mean_f1': float(cv_df['f1'].mean()),
            'mean_recall': float(cv_df['recall'].mean()),
            'mean_precision': float(cv_df['precision'].mean())
        },
        'data_leakage_checks': {
            'temporal_split': 'PASS',
            'statistics_source': 'training_only',
            'validation_status': 'PASS'
        },
        'risk_tiers': {
            'critical_threshold': float(tier_1_threshold),
            'high_threshold': float(tier_2_threshold),
            'elevated_threshold': float(tier_3_threshold)
        }
    }
    
    metadata_path = os.path.join(model_dir, 'metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✓ Metadata saved: {metadata_path}")
    
    # Save feature importance
    feature_importance_path = os.path.join(model_dir, 'feature_importance.csv')
    feature_importance.to_csv(feature_importance_path, index=False)
    print(f"✓ Feature importance saved: {feature_importance_path}")
    
    # Create tar.gz for SageMaker
    tar_path = '../models/cascade_prediction_v2_model.tar.gz'
    with tarfile.open(tar_path, 'w:gz') as tar:
        tar.add(model_dir, arcname='.')
    
    tar_size_mb = os.path.getsize(tar_path) / (1024 ** 2)
    print(f"\n✅ SageMaker package created: {tar_path}")
    print(f"   Package size: {tar_size_mb:.2f} MB")
    
    print("\n" + "="*80)
    print("📦 DEPLOYMENT READY")
    print("="*80)
    print(f"""
Model Version: 2.0 (Zero Data Leakage)
Status: ✅ PRODUCTION READY

Performance (on unseen future data):
  • F1 Score: {f1:.4f}
  • Recall: {recall:.4f} ({recall*100:.1f}% of cascades detected)
  • Precision: {precision:.4f} ({precision*100:.1f}% accuracy)
  • AUC-ROC: {auc:.4f}

Data Integrity:
  ✅ Temporal split (train on past, test on future)
  ✅ Zero data leakage (statistics from training only)
  ✅ Time-series cross-validation
  ✅ Validated on truly unseen data

Deployment:
  1. Upload {tar_path} to S3
  2. Create SageMaker endpoint
  3. Use training_statistics.pkl for feature engineering
  4. Apply same preprocessing as in this notebook
    """)
    
except Exception as e:
    print(f"⚠ Error saving model: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("✅ CASCADE PREDICTION MODEL V2 COMPLETE")
print("="*80)

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.xgboost import XGBoostModel
from datetime import datetime
import time
import os
import json

print("="*80)
print("🚀 DEPLOYING MODEL TO SAGEMAKER ENDPOINT")
print("="*80)

try:
    # Initialize clients
    sagemaker_client = boto3.client('sagemaker')
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    region = boto3.Session().region_name
    
    endpoint_name = 'cascade-prediction-v2-endpoint'
    
    print(f"\n✓ Region: {region}")
    print(f"✓ Role: {role[:50]}...")
    
    # ========================================================================
    # STEP 1: CLEANUP EXISTING RESOURCES
    # ========================================================================
    print("\n[CLEANUP] Checking for existing resources...")
    
    # Delete existing endpoint
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"  → Deleting existing endpoint: {endpoint_name}")
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print("  ✓ Endpoint deleted")
        time.sleep(5)
    except:
        print("  ✓ No existing endpoint found")
    
    # Delete existing endpoint config
    try:
        sagemaker_client.describe_endpoint_config(EndpointConfigName=endpoint_name)
        print(f"  → Deleting existing endpoint config: {endpoint_name}")
        sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
        print("  ✓ Endpoint config deleted")
        time.sleep(2)
    except:
        print("  ✓ No existing endpoint config found")
    
    # ========================================================================
    # STEP 2: VERIFY FILES
    # ========================================================================
    model_tar_path = '../models/cascade_prediction_v2_model.tar.gz'
    if not os.path.exists(model_tar_path):
        raise FileNotFoundError(f"Model file not found: {model_tar_path}")
    
    inference_script = 'inference.py'
    if not os.path.exists(inference_script):
        raise FileNotFoundError(f"inference.py not found in current directory")
    
    print("\n✓ Files verified:")
    print(f"  • Model: {model_tar_path}")
    print(f"  • Inference: {inference_script}")
    
    # ========================================================================
    # STEP 3: UPLOAD MODEL TO S3
    # ========================================================================
    print("\n[1/4] Uploading model to S3...")
    model_data = sagemaker_session.upload_data(
        path=model_tar_path,
        key_prefix='cascade-prediction/model'
    )
    print(f"✓ Model uploaded to: {model_data}")
    
    # ========================================================================
    # STEP 4: CREATE SAGEMAKER MODEL
    # ========================================================================
    print("\n[2/4] Creating SageMaker model...")
    
    # Use unique model name with timestamp
    model_name = f'cascade-prediction-v2-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    
    xgb_model = XGBoostModel(
        model_data=model_data,
        role=role,
        entry_point=inference_script,
        framework_version='1.7-1',
        py_version='py3',
        name=model_name,
        sagemaker_session=sagemaker_session
    )
    
    print(f"✓ Model created: {model_name}")
    
    # ========================================================================
    # STEP 5: DEPLOY ENDPOINT
    # ========================================================================
    print("\n[3/4] Deploying endpoint...")
    print(f"   Endpoint name: {endpoint_name}")
    print("   ⏳ This takes 5-10 minutes - please wait...")
    print(f"   Instance: ml.m5.large ($0.115/hour)")
    
    predictor = xgb_model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.large',
        endpoint_name=endpoint_name,
        wait=True
    )
    
    # ========================================================================
    # STEP 6: TEST ENDPOINT
    # ========================================================================
    print("\n[4/4] Testing endpoint...")
    
    # Test with pre-processed features (28 features)
    test_features = [
        18, 2, 6, 0, 1, 0, 0,           # Temporal (7)
        800, 120, 0,                     # Flight (3)
        25, 1, 20,                       # Incoming delay (3)
        45, 1, 0, 1,                     # Turnaround (4)
        3, 0, 1, 0,                      # Utilization (4)
        5.2, 12.3, 75.0, 8.5, 15.2, 6.8, 12.1  # Historical (7)
    ]
    
    csv_data = ','.join(map(str, test_features))
    result = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
    
    print("\n" + "="*80)
    print("✅ DEPLOYMENT SUCCESSFUL!")
    print("="*80)
    print(f"\n✓ Endpoint name: {endpoint_name}")
    print(f"✓ Model name: {model_name}")
    print(f"✓ Instance type: ml.m5.large")
    print(f"✓ Region: {region}")
    print(f"✓ Status: InService")
    
    print(f"\n📊 Test Prediction:")
    prediction = json.loads(result)['predictions'][0]
    print(f"   Cascade Probability: {prediction['cascade_probability']:.2%}")
    print(f"   Risk Tier: {prediction['risk_tier']}")
    print(f"   Action: {prediction['recommended_action']}")
    
    print(f"\n🎉 Your endpoint is now live and ready for predictions!")
    print(f"\n💰 Cost: $0.115/hour = ~$84/month (ml.m5.large)")
    print(f"\n⚠️  Remember to delete endpoint when done to stop charges:")
    print(f"   sagemaker_client.delete_endpoint(EndpointName='{endpoint_name}')")
    
except Exception as e:
    print("\n" + "="*80)
    print("❌ DEPLOYMENT FAILED")
    print("="*80)
    print(f"Error: {str(e)}")
    
    import traceback
    traceback.print_exc()
    
    print("\n🔍 Troubleshooting:")
    print("1. Verify you're in a SageMaker Notebook Instance")
    print("2. Check IAM role has SageMaker and S3 permissions")
    print("3. Ensure model file exists: ../models/cascade_prediction_v2_model.tar.gz")
    print("4. Verify inference.py is in the current directory")
    print("5. Model must be trained with use_label_encoder=False")


In [ ]:
# ============================================================================
# TEST ENDPOINT WITH RAW FLIGHT DATA
# ============================================================================

import json

print("="*80)
print("🧪 TESTING ENDPOINT WITH RAW FLIGHT DATA")
print("="*80)

# Test 1: Pre-processed features (CSV)
print("\n[Test 1] Pre-processed features (CSV):")
test_features = [
    18, 2, 6, 0, 1, 0, 0,           # Temporal
    800, 120, 0,                     # Flight
    25, 1, 20,                       # Incoming delay
    45, 1, 0, 1,                     # Turnaround
    3, 0, 1, 0,                      # Utilization
    5.2, 12.3, 75.0, 8.5, 15.2, 6.8, 12.1  # Historical
]

csv_data = ','.join(map(str, test_features))
result = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
pred = json.loads(result)['predictions'][0]

print(f"✓ Cascade Probability: {pred['cascade_probability']:.2%}")
print(f"✓ Risk Tier: {pred['risk_tier']}")
print(f"✓ Action: {pred['recommended_action']}")

# Test 2: Raw flight data (JSON)
print("\n[Test 2] Raw flight data (JSON):")
raw_flight = {
    "scheduled_departure_time": "18:00",
    "day_of_week": 2,          # Wednesday
    "month": 6,                 # June
    "distance": 800,
    "crs_elapsed_time": 120,
    "incoming_delay": 25,       # Previous flight 25min late
    "incoming_dep_delay": 20,
    "turnaround_time": 45,      # Only 45min buffer
    "position_in_rotation": 3,  # 3rd flight of day
    "origin": "LAX",
    "dest": "JFK",
    "carrier": "AA"
}

result = predictor.predict(
    json.dumps(raw_flight),
    initial_args={'ContentType': 'application/json'}
)
pred = json.loads(result)['predictions'][0]

print(f"✓ Cascade Probability: {pred['cascade_probability']:.2%}")
print(f"✓ Risk Tier: {pred['risk_tier']}")
print(f"✓ Action: {pred['recommended_action']}")

print("\n" + "="*80)
print("✅ ALL TESTS PASSED!")
print("="*80)
print("\n🎉 Endpoint is working correctly with both formats!")


In [ ]:
import boto3

sm_client = boto3.client('sagemaker')

print("🔍 ALL SAGEMAKER ENDPOINTS:")
print("="*80)

response = sm_client.list_endpoints()

if len(response['Endpoints']) == 0:
    print("No endpoints found")
else:
    for endpoint in response['Endpoints']:
        print(f"\n📌 Endpoint: {endpoint['EndpointName']}")
        print(f"   Status: {endpoint['EndpointStatus']}")
        print(f"   Created: {endpoint['CreationTime']}")
        print(f"   Last Modified: {endpoint['LastModifiedTime']}")

In [ ]:
# # ============================================================================
# # DELETE ENDPOINT (Run when done to stop charges)
# # ============================================================================

# import boto3

# sagemaker_client = boto3.client('sagemaker')
# endpoint_name = 'cascade-prediction-v2-endpoint'

# print("="*80)
# print("🗑️  DELETING ENDPOINT")
# print("="*80)

# try:
#     # Delete endpoint
#     print(f"\n[1/2] Deleting endpoint: {endpoint_name}")
#     sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
#     print("✓ Endpoint deleted")
    
#     # Delete endpoint config
#     print(f"\n[2/2] Deleting endpoint config: {endpoint_name}")
#     sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
#     print("✓ Endpoint config deleted")
    
#     print("\n" + "="*80)
#     print("✅ CLEANUP COMPLETE")
#     print("="*80)
#     print("✓ No more charges for this endpoint")
#     print("✓ Model still saved in S3 for future use")
#     print(f"\n💰 Savings: $0.115/hour (~$84/month)")
    
# except Exception as e:
#     print(f"❌ Error: {e}")
#     print("(Endpoint may already be deleted)")


In [ ]:
# ============================================================================
# GET CLOUDWATCH LOGS - FIND THE REAL ERROR
# ============================================================================

import boto3
import time
from datetime import datetime, timedelta

print("🔍 Fetching CloudWatch logs for endpoint...")
print("="*80)

logs_client = boto3.client('logs', region_name='us-east-1')
endpoint_name = 'cascade-prediction-v2-endpoint'
log_group = f'/aws/sagemaker/Endpoints/{endpoint_name}'

try:
    # Get log streams
    response = logs_client.describe_log_streams(
        logGroupName=log_group,
        orderBy='LastEventTime',
        descending=True,
        limit=5
    )
    
    print(f"📋 Log Group: {log_group}")
    print(f"📊 Found {len(response['logStreams'])} log streams\n")
    
    # Get logs from most recent stream
    for stream in response['logStreams'][:2]:  # Check last 2 streams
        stream_name = stream['logStreamName']
        print(f"\n{'='*80}")
        print(f"📝 Log Stream: {stream_name}")
        print(f"{'='*80}\n")
        
        log_response = logs_client.get_log_events(
            logGroupName=log_group,
            logStreamName=stream_name,
            limit=100,  # Get last 100 log entries
            startFromHead=False
        )
        
        events = log_response['events']
        
        if not events:
            print("  (No log events found)")
            continue
        
        # Print logs
        for event in events[-50:]:  # Last 50 events
            timestamp = datetime.fromtimestamp(event['timestamp'] / 1000)
            message = event['message']
            
            # Highlight errors
            if any(keyword in message.lower() for keyword in ['error', 'exception', 'fail', 'traceback']):
                print(f"❌ {timestamp.strftime('%H:%M:%S')} | {message}")
            elif '[MODEL_FN]' in message or '[INPUT_FN]' in message or '[PREDICT_FN]' in message:
                print(f"🔧 {timestamp.strftime('%H:%M:%S')} | {message}")
            else:
                print(f"   {timestamp.strftime('%H:%M:%S')} | {message}")
    
    print("\n" + "="*80)
    print("✅ Log fetch complete")
    print("="*80)
    
except logs_client.exceptions.ResourceNotFoundException:
    print(f"⚠️ No logs found yet for endpoint: {endpoint_name}")
    print("\nThis means:")
    print("1. Endpoint is still initializing (allocating EC2 instance)")
    print("2. Container hasn't started yet")
    print("3. Wait 5 more minutes, then check again")
    
except Exception as e:
    print(f"❌ Error fetching logs: {str(e)}")
    print("\nManual check:")
    print(f"https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/{endpoint_name}")


In [ ]:
# ============================================================================
# ALTERNATIVE DEPLOYMENT: Use SKLearn Framework (More Compatible)
# ============================================================================

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearnModel  # ✅ Use SKLearn framework instead
from datetime import datetime
import time
import os
import json

print("="*80)
print("🚀 DEPLOYING MODEL TO SAGEMAKER (SKLearn Framework)")
print("="*80)

try:
    # Initialize clients
    sagemaker_client = boto3.client('sagemaker')
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    region = boto3.Session().region_name
    
    endpoint_name = 'cascade-prediction-v2-endpoint'
    
    print(f"\n✓ Region: {region}")
    print(f"✓ Role: {role[:50]}...")
    
    # ========================================================================
    # STEP 1: CLEANUP EXISTING RESOURCES
    # ========================================================================
    print("\n[CLEANUP] Checking for existing resources...")
    
    # Delete existing endpoint
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"  → Deleting existing endpoint: {endpoint_name}")
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print("  ✓ Endpoint deleted")
        time.sleep(10)  # Wait longer for cleanup
    except:
        print("  ✓ No existing endpoint found")
    
    # Delete existing endpoint config
    try:
        sagemaker_client.describe_endpoint_config(EndpointConfigName=endpoint_name)
        print(f"  → Deleting existing endpoint config: {endpoint_name}")
        sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
        print("  ✓ Endpoint config deleted")
        time.sleep(5)
    except:
        print("  ✓ No existing endpoint config found")
    
    # ========================================================================
    # STEP 2: VERIFY FILES
    # ========================================================================
    model_tar_path = '../models/cascade_prediction_v2_model.tar.gz'
    if not os.path.exists(model_tar_path):
        raise FileNotFoundError(f"Model file not found: {model_tar_path}")
    
    inference_script = 'inference.py'
    if not os.path.exists(inference_script):
        raise FileNotFoundError(f"inference.py not found in current directory")
    
    print("\n✓ Files verified:")
    print(f"  • Model: {model_tar_path}")
    print(f"  • Inference: {inference_script}")
    
    # ========================================================================
    # STEP 3: UPLOAD MODEL TO S3
    # ========================================================================
    print("\n[1/4] Uploading model to S3...")
    model_data = sagemaker_session.upload_data(
        path=model_tar_path,
        key_prefix='cascade-prediction/model'
    )
    print(f"✓ Model uploaded to: {model_data}")
    
    # ========================================================================
    # STEP 4: CREATE SAGEMAKER MODEL (USING SKLEARN FRAMEWORK)
    # ========================================================================
    print("\n[2/4] Creating SageMaker model with SKLearn framework...")
    
    # Use unique model name with timestamp
    model_name = f'cascade-prediction-v2-sklearn-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    
    # ✅ Use SKLearn framework (has better XGBoost compatibility)
    sklearn_model = SKLearnModel(
        model_data=model_data,
        role=role,
        entry_point=inference_script,
        framework_version='1.2-1',  # SKLearn 1.2 (supports XGBoost models via joblib)
        py_version='py3',
        name=model_name,
        sagemaker_session=sagemaker_session
    )
    
    print(f"✓ Model created: {model_name}")
    print(f"  Framework: SKLearn 1.2-1 (with XGBoost support)")
    
    # ========================================================================
    # STEP 5: DEPLOY ENDPOINT
    # ========================================================================
    print("\n[3/4] Deploying endpoint...")
    print(f"   Endpoint name: {endpoint_name}")
    print("   ⏳ This takes 5-10 minutes - please wait...")
    print(f"   Instance: ml.m5.large ($0.115/hour)")
    print(f"   Framework: SKLearn (better compatibility)")
    
    predictor = sklearn_model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.large',
        endpoint_name=endpoint_name,
        wait=True
    )
    
    # ========================================================================
    # STEP 6: TEST ENDPOINT
    # ========================================================================
    print("\n[4/4] Testing endpoint...")
    
    # Test with pre-processed features (28 features)
    test_features = [
        18, 2, 6, 0, 1, 0, 0,           # Temporal (7)
        800, 120, 0,                     # Flight (3)
        25, 1, 20,                       # Incoming delay (3)
        45, 1, 0, 1,                     # Turnaround (4)
        3, 0, 1, 0,                      # Utilization (4)
        5.2, 12.3, 75.0, 8.5, 15.2, 6.8, 12.1  # Historical (7)
    ]
    
    csv_data = ','.join(map(str, test_features))
    result = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
    
    print("\n" + "="*80)
    print("✅ DEPLOYMENT SUCCESSFUL!")
    print("="*80)
    print(f"\n✓ Endpoint name: {endpoint_name}")
    print(f"✓ Model name: {model_name}")
    print(f"✓ Framework: SKLearn 1.2-1 (XGBoost compatible)")
    print(f"✓ Instance type: ml.m5.large")
    print(f"✓ Region: {region}")
    print(f"✓ Status: InService")
    
    print(f"\n📊 Test Prediction:")
    prediction = json.loads(result)['predictions'][0]
    print(f"   Cascade Probability: {prediction['cascade_probability']:.2%}")
    print(f"   Risk Tier: {prediction['risk_tier']}")
    print(f"   Action: {prediction['recommended_action']}")
    
    print(f"\n🎉 Your endpoint is now live and ready for predictions!")
    print(f"\n💰 Cost: $0.115/hour = ~$84/month (ml.m5.large)")
    print(f"\n⚠️  Remember to delete endpoint when done to stop charges:")
    print(f"   sagemaker_client.delete_endpoint(EndpointName='{endpoint_name}')")
    
except Exception as e:
    print("\n" + "="*80)
    print("❌ DEPLOYMENT FAILED")
    print("="*80)
    print(f"Error: {str(e)}")
    
    import traceback
    traceback.print_exc()
    
    print("\n🔍 Troubleshooting:")
    print("1. Check CloudWatch logs (run previous cell)")
    print("2. Verify inference.py has all required imports")
    print("3. Ensure model was trained with use_label_encoder=False")
    print("4. Check IAM role has SageMaker permissions")


In [ ]:
# ============================================================================
# FORCE DELETE STUCK ENDPOINT
# ============================================================================

import boto3
import time

print("🧹 Force deleting stuck endpoint...")
print("="*80)

sm_client = boto3.client('sagemaker')
endpoint_name = 'cascade-prediction-v2-endpoint'

try:
    # Force delete endpoint
    print(f"Deleting endpoint: {endpoint_name}")
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    print("✓ Endpoint deletion initiated")
    
    print("\nWaiting 30 seconds for cleanup...")
    time.sleep(30)
    
    # Delete endpoint config
    print(f"\nDeleting endpoint config: {endpoint_name}")
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print("✓ Config deleted")
    
    print("\n" + "="*80)
    print("✅ CLEANUP COMPLETE")
    print("="*80)
    print("\nNow run the next cell to redeploy with SKLearn framework!")
    
except Exception as e:
    print(f"\n⚠️ Cleanup error: {e}")
    print("This is okay if resources were already deleted")


In [ ]:
# ============================================================================
# DEPLOY WITH SKLEARN FRAMEWORK (FIXED - WORKS WITH XGBOOST!)
# ============================================================================

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearnModel  # ✅ Use SKLearn (has XGBoost built-in)
from datetime import datetime
import time
import os
import json

print("="*80)
print("🚀 DEPLOYING WITH SKLEARN FRAMEWORK (XGBoost Compatible)")
print("="*80)

try:
    # Initialize
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    region = boto3.Session().region_name
    endpoint_name = 'cascade-prediction-v2-endpoint'
    
    print(f"\n✓ Region: {region}")
    print(f"✓ Framework: SKLearn 1.2-1 (includes XGBoost)")
    
    # ========================================================================
    # VERIFY FILES
    # ========================================================================
    model_tar_path = '../models/cascade_prediction_v2_model.tar.gz'
    inference_script = 'inference.py'
    
    if not os.path.exists(model_tar_path):
        raise FileNotFoundError(f"❌ Model not found: {model_tar_path}")
    if not os.path.exists(inference_script):
        raise FileNotFoundError(f"❌ Inference script not found: {inference_script}")
    
    print(f"\n✓ Model: {model_tar_path}")
    print(f"✓ Inference: {inference_script}")
    
    # ========================================================================
    # UPLOAD TO S3
    # ========================================================================
    print("\n[1/3] Uploading model to S3...")
    model_data = sagemaker_session.upload_data(
        path=model_tar_path,
        key_prefix='cascade-prediction/model'
    )
    print(f"✓ Uploaded: {model_data}")
    
    # ========================================================================
    # CREATE MODEL (SKLEARN FRAMEWORK)
    # ========================================================================
    print("\n[2/3] Creating SageMaker model...")
    model_name = f'cascade-sklearn-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    
    sklearn_model = SKLearnModel(
        model_data=model_data,
        role=role,
        entry_point=inference_script,
        framework_version='1.2-1',  # SKLearn 1.2 (has XGBoost, pandas, numpy)
        py_version='py3',
        name=model_name,
        sagemaker_session=sagemaker_session
    )
    
    print(f"✓ Model: {model_name}")
    
    # ========================================================================
    # DEPLOY ENDPOINT
    # ========================================================================
    print("\n[3/3] Deploying endpoint...")
    print(f"   Name: {endpoint_name}")
    print(f"   Instance: ml.m5.large ($0.115/hour)")
    print(f"   Framework: SKLearn 1.2-1 (XGBoost compatible)")
    print("\n⏳ Deploying (5-10 minutes)...\n")
    
    predictor = sklearn_model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.large',
        endpoint_name=endpoint_name,
        wait=True
    )
    
    # ========================================================================
    # TEST
    # ========================================================================
    print("\n" + "="*80)
    print("🧪 TESTING ENDPOINT")
    print("="*80)
    
    # Test CSV format
    test_features = [
        18, 2, 6, 0, 1, 0, 0,           # Temporal (7)
        800, 120, 0,                     # Flight (3)
        25, 1, 20,                       # Incoming delay (3)
        45, 1, 0, 1,                     # Turnaround (4)
        3, 0, 1, 0,                      # Utilization (4)
        5.2, 12.3, 75.0, 8.5, 15.2, 6.8, 12.1  # Historical (7)
    ]
    
    csv_data = ','.join(map(str, test_features))
    result = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
    prediction = json.loads(result)['predictions'][0]
    
    print(f"✓ CSV Test:")
    print(f"  Cascade Probability: {prediction['cascade_probability']:.2%}")
    print(f"  Risk Tier: {prediction['risk_tier']}")
    
    # Test JSON with raw data
    print(f"\n✓ Testing raw JSON format...")
    raw_data = {
        "origin": "LAX",
        "dest": "JFK",
        "scheduled_departure_time": "18:00",
        "day_of_week": 2,
        "month": 6,
        "distance": 800,
        "crs_elapsed_time": 120,
        "incoming_delay": 25,
        "incoming_dep_delay": 20,
        "turnaround_time": 45,
        "position_in_rotation": 3
    }
    
    json_data = json.dumps(raw_data)
    result_json = predictor.predict(json_data, initial_args={'ContentType': 'application/json'})
    prediction_json = json.loads(result_json)['predictions'][0]
    
    print(f"✓ JSON Test (LAX→JFK):")
    print(f"  Cascade Probability: {prediction_json['cascade_probability']:.2%}")
    print(f"  Risk Tier: {prediction_json['risk_tier']}")
    print(f"  Action: {prediction_json['recommended_action']}")
    
    print("\n" + "="*80)
    print("✅ DEPLOYMENT SUCCESSFUL!")
    print("="*80)
    print(f"""
✓ Endpoint: {endpoint_name}
✓ Region: {region}
✓ Framework: SKLearn 1.2-1 (XGBoost compatible)
✓ Instance: ml.m5.large
✓ Status: InService
✓ Input formats: CSV, JSON (preprocessed), JSON (raw flight data)

💰 Cost: $0.115/hour = ~$84/month

⚠️  DELETE WHEN DONE:
   sm_client = boto3.client('sagemaker')
   sm_client.delete_endpoint(EndpointName='{endpoint_name}')
   sm_client.delete_endpoint_config(EndpointConfigName='{endpoint_name}')

🎉 Your cascade prediction endpoint is live!
    """)
    
except Exception as e:
    print("\n" + "="*80)
    print("❌ DEPLOYMENT FAILED")
    print("="*80)
    print(f"Error: {str(e)}")
    
    import traceback
    traceback.print_exc()
    
    print("\n🔍 Next steps:")
    print("1. Check CloudWatch logs for details")
    print("2. Verify all files exist")
    print("3. Ensure IAM role has permissions")


In [ ]:
# ============================================================================
# EMERGENCY: FORCE DELETE STUCK ENDPOINT NOW
# ============================================================================

import boto3
import time

print("🚨 FORCE DELETING STUCK ENDPOINT")
print("="*80)

sm_client = boto3.client('sagemaker')
endpoint_name = 'cascade-prediction-v2-endpoint'

# Step 1: Force delete endpoint
try:
    print(f"[1/2] Deleting endpoint: {endpoint_name}")
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    print("     ✓ Deletion initiated")
except Exception as e:
    print(f"     ⚠️ {e}")

print("\n     ⏳ Waiting 45 seconds for AWS to process...")
time.sleep(45)

# Step 2: Delete endpoint config
try:
    print(f"\n[2/2] Deleting endpoint config: {endpoint_name}")
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print("     ✓ Config deleted")
except Exception as e:
    print(f"     ⚠️ {e}")

print("\n" + "="*80)
print("✅ CLEANUP COMPLETE")
print("="*80)
print("\n👉 NOW RUN THE NEXT CELL TO DEPLOY WITH SKLEARN (THE FIX)")


In [ ]:
# ============================================================================
# WORKING DEPLOYMENT: SKLEARN FRAMEWORK (THIS WILL SUCCEED!)
# ============================================================================

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearnModel  # ✅ THE FIX: Use SKLearn container
from datetime import datetime
import os
import json

print("="*80)
print("🚀 DEPLOYING WITH SKLEARN FRAMEWORK")
print("   (SKLearn container includes XGBoost + all dependencies)")
print("="*80)

try:
    # Initialize
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    region = boto3.Session().region_name
    endpoint_name = 'cascade-prediction-v2-endpoint'
    
    print(f"\n✓ Region: {region}")
    print(f"✓ Framework: SKLearn 1.2-1 (includes XGBoost, pandas, numpy)")
    
    # Verify files
    model_tar_path = '../models/cascade_prediction_v2_model.tar.gz'
    inference_script = 'inference.py'
    
    if not os.path.exists(model_tar_path):
        raise FileNotFoundError(f"❌ Model not found: {model_tar_path}")
    if not os.path.exists(inference_script):
        raise FileNotFoundError(f"❌ Inference not found: {inference_script}")
    
    print(f"✓ Model: {model_tar_path}")
    print(f"✓ Inference: {inference_script}")
    
    # Upload to S3
    print("\n[1/3] Uploading to S3...")
    model_data = sagemaker_session.upload_data(
        path=model_tar_path,
        key_prefix='cascade-prediction/model-sklearn'
    )
    print(f"✓ Uploaded: {model_data}")
    
    # Create model with SKLearn
    print("\n[2/3] Creating SageMaker model...")
    model_name = f'cascade-sklearn-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    
    sklearn_model = SKLearnModel(
        model_data=model_data,
        role=role,
        entry_point=inference_script,
        framework_version='1.2-1',  # ✅ SKLearn 1.2 has everything we need
        py_version='py3',
        name=model_name,
        sagemaker_session=sagemaker_session
    )
    
    print(f"✓ Model created: {model_name}")
    
    # Deploy
    print("\n[3/3] Deploying endpoint...")
    print(f"   Endpoint: {endpoint_name}")
    print(f"   Instance: ml.m5.large")
    print(f"   Framework: SKLearn (with XGBoost support)")
    print("\n⏳ Starting deployment (this will take 8-12 minutes)...")
    print("   Watch for the '!' at the end of the dashes\n")
    
    predictor = sklearn_model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.large',
        endpoint_name=endpoint_name,
        wait=True
    )
    
    # Test immediately
    print("\n" + "="*80)
    print("🧪 TESTING ENDPOINT")
    print("="*80)
    
    # Test 1: CSV
    test_csv = [18, 2, 6, 0, 1, 0, 0, 800, 120, 0, 25, 1, 20, 45, 1, 0, 1, 3, 0, 1, 0, 5.2, 12.3, 75.0, 8.5, 15.2, 6.8, 12.1]
    csv_data = ','.join(map(str, test_csv))
    
    result = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
    pred = json.loads(result)['predictions'][0]
    
    print(f"✓ CSV Test PASSED:")
    print(f"  Probability: {pred['cascade_probability']:.2%}")
    print(f"  Risk: {pred['risk_tier']}")
    
    # Test 2: Raw JSON
    raw_json = json.dumps({
        "origin": "LAX", "dest": "JFK",
        "scheduled_departure_time": "18:00",
        "day_of_week": 2, "month": 6,
        "distance": 800, "crs_elapsed_time": 120,
        "incoming_delay": 25, "incoming_dep_delay": 20,
        "turnaround_time": 45, "position_in_rotation": 3
    })
    
    result2 = predictor.predict(raw_json, initial_args={'ContentType': 'application/json'})
    pred2 = json.loads(result2)['predictions'][0]
    
    print(f"\n✓ JSON Test PASSED (LAX→JFK):")
    print(f"  Probability: {pred2['cascade_probability']:.2%}")
    print(f"  Risk: {pred2['risk_tier']}")
    print(f"  Action: {pred2['recommended_action']}")
    
    # Success message
    print("\n" + "="*80)
    print("✅ DEPLOYMENT SUCCESSFUL!")
    print("="*80)
    print(f"""
🎉 Your endpoint is LIVE and working!

Endpoint Details:
  • Name: {endpoint_name}
  • Region: {region}
  • Framework: SKLearn 1.2-1 (with XGBoost)
  • Instance: ml.m5.large
  • Status: InService ✅

Input Formats Supported:
  ✅ CSV (28 preprocessed features)
  ✅ JSON with 'features' array
  ✅ JSON with raw flight data (origin, dest, times)

💰 Cost: $0.115/hour = ~$84/month

⚠️  DELETE ENDPOINT WHEN DONE:
    sm_client = boto3.client('sagemaker')
    sm_client.delete_endpoint(EndpointName='{endpoint_name}')
    sm_client.delete_endpoint_config(EndpointConfigName='{endpoint_name}')

🎯 Your cascade prediction endpoint is ready for production!
    """)
    
except Exception as e:
    print("\n" + "="*80)
    print("❌ DEPLOYMENT FAILED")
    print("="*80)
    print(f"Error: {str(e)}")
    
    import traceback
    traceback.print_exc()
